In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [19]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

In [20]:
!pip install langchain-text-splitters

In [21]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

#1-INDEXING

1-A DocumentLoader (Indexing)

In [22]:
#Fet

video_id = "Gfr50f6ZBvo"  # only the ID, not full URL
try:
    # Fetch transcript directly (returns FetchedTranscriptSnippet objects)
    transcript = YouTubeTranscriptApi().fetch(video_id, languages=['en'])

    # Flatten into plain text
    text = " ".join(snippet.text for snippet in transcript)
    print(text[:500])  # print first 500 characters for preview

except TranscriptsDisabled:
    print("No captions available for this video.")

the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful 


1-B Text-Splitting (Indexing)

In [23]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

chunks = splitter.split_text(text)

In [24]:
len(chunks)

168

In [25]:
chunks[0]

"the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough

1-C Embeddings Generation and 1-D Vector-Store

In [26]:
from langchain_core.documents import Document
documents = [Document(page_content=chunk) for chunk in chunks]

In [27]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")
vector_store = FAISS.from_documents(documents,embeddings)

In [28]:
vector_store.index_to_docstore_id

{0: '33ed6511-1445-49cd-8274-e578a313de83',
 1: 'b0ef2c51-a118-48df-8c19-86a30ff7f87b',
 2: '15e87742-0619-42bf-aa82-d0b5aae31fe6',
 3: '711ba499-be37-4ff7-8256-e9598b4d4deb',
 4: '59a6a686-f1e0-430b-9493-df6ef45aaea1',
 5: 'da6e8526-ff1d-418e-b110-ce54ccfe5e98',
 6: '3e8ebbf3-1d9f-4761-8f43-76ecd0a11e64',
 7: '57a9e800-badf-49b0-af83-4141f2a7a1b7',
 8: 'd1e7958c-86a2-49f6-abed-080af9dc06ae',
 9: '5c2eb9de-d52e-4eb1-9766-a053d8f9dffc',
 10: '80e4d1c3-9a19-426b-8df4-ba4b349ec8a6',
 11: '5f87d112-1720-4e40-87a9-a829db29c916',
 12: 'd69b179b-e361-40b0-bdf2-838c274c02b5',
 13: '5c1fbabf-fa83-4bcc-bd06-18792cdfb20c',
 14: 'cc194769-23c9-4271-adf8-28d95cb95195',
 15: '4440fae6-7504-45ba-b26f-ddad77fb410f',
 16: 'aee8d464-6105-4828-b896-a508ddbe5038',
 17: '4c79b8c7-2ad3-40ff-a0e8-c4e8e0e7f703',
 18: 'e849726d-3abc-4029-82ab-d330cb3f5f8d',
 19: 'e85b4fd9-5cdc-4cd1-8da9-e24ec2f7f766',
 20: '02c17981-cdd1-48e7-abf2-136f14047520',
 21: 'a99ec53d-65d0-4b75-bc4f-57344d9e4fae',
 22: '0dd56a76-5478-

In [ ]:
vector_store.get_by_ids(['a4a87ead-9d95-4604-8f3a-607ba45d5c27'])                        

[Document(id='a4a87ead-9d95-4604-8f3a-607ba45d5c27', metadata={}, page_content='demas establish to support this podcast please check out our sponsors in the description and now let me leave you with some words from edskar dykstra computer science is no more about computers than astronomy is about telescopes thank you for listening and hope to see you next time')]

Retriever

In [30]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [31]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1725b6910>, search_kwargs={'k': 4})

In [32]:
retriever.invoke("What is deep mind?")

[Document(id='cbeeb2ca-d817-48eb-ab2d-16c6bb22031b', metadata={}, page_content='i used to discuss um uh uh what were the sort of founding tenets of deep mind and it was very various things one was um algorithmic advances so deep learning you know jeff hinton and cohen just had just sort of invented that in academia but no one in industry knew about it uh we love reinforcement learning we thought that could be scaled up but also understanding about the human brain had advanced um quite a lot uh in the decade prior with fmri machines and other things so we could get some good hints about architectures and algorithms and and sort of um representations maybe that the brain uses so as at a systems level not at a implementation level um and then the other big things were compute and gpus right so we could see a compute was going to be really useful and it got to a place where it became commoditized mostly through the games industry and and that could be taken advantage of and then the final 

Augmentation

In [33]:
llm = ChatOpenAI(model="gpt-5-nano-2025-08-07", temperature=0.2)

In [52]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't get the Info.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
question = "Is topic of alien is discussed in the video? If yes, then what was discussed?"
retrieved_docs = retriever.invoke(question)

In [54]:
retrieved_docs

[Document(id='231e8a6f-8293-4a27-9a67-bebfa821b088', metadata={}, page_content="thoughts it could be some interactions with our mind that we think are originating from us is actually something that uh is coming from other life forms elsewhere consciousness itself might be that it could be but i don't see any sensible argument to the why why would all of the alien species be using this way yes some of them will be more primitive they would be close to our level you know there would there should be a whole sort of normal distribution of these things right some would be aggressive some would be you know curious others would be very stoical and philosophical because you know maybe they're a million years older than us but it's not it shouldn't be like what i mean one one alien civilization might be like that communicating thoughts and others but i don't see why you know potentially the hundreds there should be would be uniform in this way right it could be a violent dictatorship that the t

In [55]:
context_text = "\n".join([doc.page_content for doc in retrieved_docs])
context_text

"thoughts it could be some interactions with our mind that we think are originating from us is actually something that uh is coming from other life forms elsewhere consciousness itself might be that it could be but i don't see any sensible argument to the why why would all of the alien species be using this way yes some of them will be more primitive they would be close to our level you know there would there should be a whole sort of normal distribution of these things right some would be aggressive some would be you know curious others would be very stoical and philosophical because you know maybe they're a million years older than us but it's not it shouldn't be like what i mean one one alien civilization might be like that communicating thoughts and others but i don't see why you know potentially the hundreds there should be would be uniform in this way right it could be a violent dictatorship that the the people the alien civilizations that uh become successful become um [Music]\n

In [56]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [57]:
print(final_prompt)

text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't get the Info.\n\n      thoughts it could be some interactions with our mind that we think are originating from us is actually something that uh is coming from other life forms elsewhere consciousness itself might be that it could be but i don't see any sensible argument to the why why would all of the alien species be using this way yes some of them will be more primitive they would be close to our level you know there would there should be a whole sort of normal distribution of these things right some would be aggressive some would be you know curious others would be very stoical and philosophical because you know maybe they're a million years older than us but it's not it shouldn't be like what i mean one one alien civilization might be like that communicating thoughts and others but i don't see why you know potentially the hund

Generation

In [58]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes. The video discusses aliens and alien civilizations, including how they might communicate, how many civilizations could exist, and whether humanity is alone.


Now above we had Implemented the Project without Chains which is looking very unsystemetic

Building Chians

In [60]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
    context_text = "\n".join([doc.page_content for doc in retrieved_docs]) # concatenate document contents
    return context_text

In [62]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [67]:
parallel_chain.invoke("Who is Demus?")

{'context': "the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get

In [68]:
parser = StrOutputParser()

In [69]:
main_chian = parallel_chain | prompt | llm | parser

In [72]:
main_chian.invoke("Can you summarize the video?")

'- The conversation explores the idea that there may be deeper, more fundamental explanations of physics beyond the standard model, offering glimpses of how things like consciousness, life, and gravity might be understood in a simpler or more elaborate framework.\n- It features a tribute to Demas and discusses starting points for explaining big mysteries, emphasizing the possibility of new approaches that could change how we think about physics.\n- On artificial intelligence, it asks how much of solving intelligence is science versus engineering—covering algorithms, data, hardware/compute, software/infrastructure, human factors, and philosophy—and suggests the key to long-term progress is a combination of all these elements, not one single factor.\n- The dialogue touches on broader philosophical ideas: the “safari view” of humanity, climate challenges, and the simulation hypothesis, arguing that our experience might be a projection from a deeper, computational underlying reality.'